In [ ]:
# !pip install -U datasets

In [ ]:
# !sudo -H pip install -q transformers --upgrade

In [ ]:
# !pip uninstall autoawq -y

In [ ]:
# !pip install autoawq

In [ ]:
# !sudo -H pip install https://github.com/casper-hansen/AutoAWQ/releases/download/v0.1.7/autoawq-0.1.7+cu118-cp38-cp38-linux_x86_64.whl

In [ ]:
# !pip install transformers==4.35.2

In [2]:
import pandas
import numpy
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:512"

In [3]:
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer,AutoModelForCausalLM

In [4]:
import torch
torch.__version__

'2.0.1+cu117'

In [ ]:
#model_path = '/data/mistral/query-to-mql/exp-10/nov-20/merged-model'

In [5]:
quant_path = '/llmmodels/quantized_model/awq_mistral'

In [6]:
model_path='mistralai/Mistral-7B-v0.1'

In [7]:
quant_config = {"zero_point":True,
               "q_group_size": 128,
               "w_bit": 4,
               "version": "GEMM"
               }

In [8]:
#load model
# model = AutoAWQForCausalLM.from_pretrained(model_path, **{"low_cpu_mem_usage":True})
model = AutoAWQForCausalLM.from_pretrained(model_path, device_map='auto')

Fetching 11 files:   0%|          | 0/11 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

In [10]:
import torch
torch.cuda.empty_cache()

In [11]:
import gc
gc.collect()


31

In [ ]:
# torch.cuda.memory_summary(device=None, abbreviated=False)

In [ ]:
#Quantize
# dont run it again
model.quantize(tokenizer, quant_config=quant_config)

/tmp/pip_packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")
AWQ:  84%|████████▍ | 27/32 [34:44<06:17, 75.54s/it]

In [ ]:
# model_quantized = AutoModelForCausalLM.from_pretrained(
#     model,
#     torch_dtype=torch.float16,
#     device_map="auto",
#     quantization_config=quant_config
# )

In [ ]:
model.save_quantized(quant_path)
tokenizer.save_pretrained(quant_path)

### Inference on quantized model

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
quant_path = '/data/quantization-trials/quant-model'
model_path = '/data/quantization-trials/merged-model'

In [ ]:
model_id = quant_path
quant_model = AutoModelForCausalLM.from_pretrained(model_id, device_map="cuda")
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

def predict_from_quant(user_query):
    _inputs = tokenizer.encode(user_query, return_tensors="pt").to('cuda')
    outputs = quant_model.generate(input_ids=_inputs, max_length= 1000, pad_token_id=tokenizer.eos_token_id)
    output = tokenizer.decode(outputs[0])
    return output

In [ ]:
import time

In [ ]:
# Using quant model
start = time.time()
output1 = predict_from_quant("what is art")
print("time taken is :", time.time()-start)

In [ ]:
print(output1)

### Inference on un-quantized model

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
model_path = '/data/quantization-trials/merged-model'

In [ ]:
model_id = model_path
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="cuda")
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)

def predict_from_normal(user_query):
    _inputs = tokenizer.encode(user_query, return_tensors="pt").to('cuda')
    outputs = model.generate(input_ids=_inputs, max_length= 1000, pad_token_id=tokenizer.eos_token_id)
    output = tokenizer.decode(outputs[0])
    return output

In [ ]:
# Using unquant model
import time
start = time.time()
output2 = predict_from_normal("what is art")
print("time taken is :", time.time()-start)

In [ ]:
print(output2)

In [ ]:
model

In [ ]:
quant_model